In [2]:
import os
from glob import glob

import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import box

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
root_path = "/home/fred/projects/open_gira/open-gira/"

# manually created targets file with country iso codes
#targets_path = os.path.join(root_path, "results/power/targets.geoparquet")
targets_path = os.path.join(root_path, "results/power/targets_with_iso_a3.geoparquet")

# copying old file while creating new one -- 20230303T181600
#exposure_path = os.path.join(root_path, "results/power/by_storm_set/IBTrACS/exposure_by_target.nc")
exposure_path = os.path.join(root_path, "exposure_by_target.nc")

tracks_path = os.path.join(root_path, "results/input/IBTrACS/processed/v4.geoparquet")

validation_data_path = os.path.join(root_path, "validation/outage_model_validation.csv")

In [7]:
targets = gpd.read_parquet(targets_path)
exposure = xr.open_dataset(exposure_path)
borders = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
tracks = gpd.read_parquet(tracks_path)
validation_data = pd.read_csv(validation_data_path)

In [ ]:
plot_dir = os.path.join("validation", "plots", "target_status")
os.makedirs(plot_dir, exist_ok=True)

events_to_plot = sorted(set(validation_data.event_id) & set(exposure.event_id.values))
#events_to_plot = sorted(set(exposure.event_id.values))

for event_id in events_to_plot:

    # subset tracks
    track = tracks[tracks.track_id == event_id]

    # find all targets within 3 degrees of track
    track_bbox = box(*track.geometry.total_bounds)
    aoi = track_bbox.buffer(3)
    aoi_targets = targets[targets.within(aoi)]

    for threshold in exposure.threshold.values:
        
        threshold_str = f"{threshold:.2f}".replace(".", "p")
        plot_filepath = os.path.join(plot_dir, f"{event_id}_{threshold_str}.png")
        
        if not os.path.exists(plot_filepath):

            try:
                df = exposure.supply_factor.sel(dict(event_id=event_id, threshold=threshold))
                df = df.to_dataframe().reset_index()[["target", "supply_factor"]]
                df = df.rename(columns={"target": "id"})

                # drop targets with NaN supply_factor
                df = df[~df.supply_factor.isna()]

                # combine target information with exposure
                data = gpd.GeoDataFrame(df.merge(aoi_targets, how="inner", on="id"))
                data.geometry = data.geometry.centroid

                # categorise supply_factor
                status_cmap = {"Disconnected": "firebrick", "Degraded": "salmon", "Nominal": "lightgrey", "Oversupply": "darkorchid"}
                data["connection_status"] = pd.cut(
                    data.supply_factor,
                    bins=[-1E3, 0.25, 0.90, 1.1, 1E3],
                    labels=status_cmap.keys()
                )
                data["colour"] = data.connection_status.map(status_cmap)

                f, ax = plt.subplots(figsize=(8,8))

                # plot landmasses and political borders
                borders.plot(ax=ax, facecolor="none", edgecolor="grey", alpha=0.5)

                # plot supply_factor
                ax.scatter(data.geometry.x, data.geometry.y, c=data.colour, alpha=0.5, marker="o", s=1)
                handles = [
                    Line2D([0], [0], marker='o', color='w', markerfacecolor=v, label=k, markersize=8)
                    for k, v in status_cmap.items() if isinstance(k, str)
                ]
                ax.legend(handles=handles, ncol=1, title="Node status")

                # plot tracks with colourbar for wind speed intensity
                markersize = np.exp(track.category)
                divider = make_axes_locatable(ax)
                cax = divider.append_axes("right", size="3%", pad=0.01)
                track.plot(column="max_wind_speed_ms", ax=ax, cax=cax, s=markersize, alpha=0.2, legend=True)
                cax.set_ylabel("Wind speed $[m s^{-1}]$")

                # set window to AOI (track with a buffer)
                min_x, min_y, max_x, max_y = aoi.bounds
                ax.set_xlim(min_x, max_x)
                ax.set_ylim(min_y, max_y)
                ax.set_xlabel("Longitude [deg]")
                ax.set_ylabel("Latitude [deg]")
                ax.grid()

                name, = set(track.name)
                year, = set(track.year)
                ax.set_title(f"{event_id}: {name}, {year:d} @ {threshold:.1f} $[m s^{{-1}}]$")

                f.savefig(plot_filepath)

            except:
                # skip if we failed
                continue

/tmp/ipykernel_1495182/359583426.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data.geometry = data.geometry.centroid
/tmp/ipykernel_1495182/359583426.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data.geometry = data.geometry.centroid
/tmp/ipykernel_1495182/359583426.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data.geometry = data.geometry.centroid
/tmp/ipykernel_1495182/359583426.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS b

/tmp/ipykernel_1495182/359583426.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data.geometry = data.geometry.centroid
/tmp/ipykernel_1495182/359583426.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data.geometry = data.geometry.centroid
/tmp/ipykernel_1495182/359583426.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data.geometry = data.geometry.centroid
/tmp/ipykernel_1495182/359583426.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS b